<center>
    
# Plotting the shear profile of the E06 floating lidar data utilising the WRA Data Model and the brightwind library
## by
## Stephen Holleran (BrightWind), 26-Oct-2023

</center>


This is a notebook demonstrating some of the benefits of the Task 43 WRA Data Model using it with real data and the brightwind Python library. The brightwind library is an open-source Python library specifically developed for processing wind resource data. It can be installed using `pip install brightwind` and it's GitHub repo here: https://github.com/brightwind-dev/brightwind

The timeseries data used here is publicly available for download at: https://oswbuoysny.resourcepanorama.dnv.com/.
The WRA Data Model formatted JSON file for the E06 floating lidar was created by Daniel Nuno of Eolos and can be found in the 'demo_data' folder of this repository.

This Notebook builds on the [Notebook](https://github.com/IEA-Task-43/digital_wra_data_standard/blob/e590563e9f6d6987c5f267055f0b542be0970f2e/tools/2023%20Workshop%20-%20Floating%20Lidar%20Demo%20-%20Daniel%20Nuno.ipynb) presented by Daniel Nuno (Eolos) during the [2023 Workshop](https://github.com/IEA-Task-43/digital_wra_data_standard/wiki/Presentations).

In [ ]:
import brightwind as bw
import pandas as pd
import pprint as pp

## Load in WRA Data Model file

Create a measurement station object using the E06 floating lidar WRA Data Model formatted JSON file produced by Eolos.

In [ ]:
e06_data_model_file = '../demo_data/E06_wraMetaData.json' # this file is located in the 'demo_data' folder of the GitHub repository.
meas_sta = bw.MeasurementStation(e06_data_model_file)

### Show the measurement station's high level info along with the vertical profiler properties

In [ ]:
meas_sta.get_table()

The Name and Lat, Long values can be seen in the top half of the table. The bottom half contains the two vertical profiler
properties used in this measurement campaign. One is for the lidar itself and the other is for an ADCP which is mounted underneath the buoy pointing down towards the sea floor.

Note the `Device Datum Plane Height [m]` for the lidar is stated as 1.6 m above mean sea level.

### Show all the logger configurations that exist for this station

In [ ]:
meas_sta.logger_main_configs.get_table()

### Show all the measurements been made by this station

In [ ]:
meas_sta.measurements.get_table()

There are far to many rows for the dataframe to display so we can filter for just the horizontal wind speeds.

In [ ]:
meas_sta.measurements.get_table(wind_speeds=True)

You can see all the lidar measurements start with the highest and working down in elevation. The last one is the onboard met station that is mounted on the buoy itself.

## Load the timeseries data

In [ ]:
data = bw.load_csv('E06_Hudson_South_10_min_avg_20190904_20220327.csv', engine='python').apply(pd.to_numeric, 
                                                                                               errors='coerce')
data.columns = data.columns.str.strip()

Normally loading a csv into the brightwind library would be as simple as `bw.load_csv('filename.csv')`. However, due to poor formatting of the timeseries data file downloaded directly from the above link there are few extra things to do to read this in properly.
1. `engine='python'` is added due to the 3 leading character spaces before the 'NaN' values, Pandas throws an error as it thinks there are multiple data types, strings and numeric in the one column.
1. Also, due to this we need to force this to a numeric by using `.apply(pd.to_numeric, errors='coerce')`
1. Finally, the column names for some reason have a leading space which needs to be stripped out using `data.columns.str.strip()`. 

Thanks to Daniel Nuno of Eolos for solving these issues.

In [ ]:
data

### Get the wind speeds and related heights from the data model
The brightwind recommended method to assemble raw timeseries data files is to use the measurement point name as the 
column name in the resulting assembled file and not the raw column names. The raw column names can mean different things throughout a full measurement campaign. For example a raw column name of 'Ch1_Avg' may be wired to an anemometer mounted at 80 m. However, half way through the campaign a 60 m anemometer was wired to 'Ch1_Avg' instead. An assembled data file using 'Ch1_Avg' as the column name would now have wind speeds from 80 m and then switching to 60 m. The analyst would need to split this column of data. Using the measurement point name instead when assembling the daily files will already account for this splitting and provide a data file with a column name, for example, of 'Wspd_80m'. This could be used by the analyst immediately without any further splitting and rejigging.

To account for this, the function below will find the correct column name and associated measurement heights which can then be used in the shear plot.

In [ ]:
def get_wspd_avg_cols_and_heights(measurements_data_model, min_height=0, max_height=1000):
    '''
    Get the wind speed average column names and associated heights.
    Can filter for a height range.
    Returns a tuple of list of column names and heights (tuple(list, list)).
    '''
    wspd_avg_col_names = []
    wspd_avg_col_names_heights = []
    for meas in measurements_data_model:
        if meas.get('measurement_type_id') == 'wind_speed' and min_height <= meas.get('height_m') <= max_height:
            wspd_avg_col_names_heights.append(meas.get('height_m'))
            for column_name in meas.get('logger_measurement_config')[0].get('column_name'):
                if column_name.get('statistic_type_id') == 'avg':
                    wspd_avg_col_names.append(column_name.get('column_name'))
    return (wspd_avg_col_names, wspd_avg_col_names_heights)

Use the function to get the raw data column name heights for wind speeds and their associated heights. This is filtered by height to drop the met station wind speed which is mounted on the buoy at 1.4 m.

In [ ]:
wspd_avg_col_names, wspd_avg_col_names_heights = get_wspd_avg_cols_and_heights(meas_sta.measurements.data_model, 
                                                                               min_height=20)

print("Wind speed data column names:")
pp.pp(wspd_avg_col_names)
print("\nAssociated wind speed heights:")
pp.pp(wspd_avg_col_names_heights)

Note that the data column name contains a height, e.g. _'lidar_lidar18m_Z10_HorizWS'_ of 18 m, which is different to the height of the measurement point. This 2 m height difference is to account for the lidar mounted on the buoy which is floating above the 'mean_sea_level' and the height of the device itself. As noted in the first table the `Device Datum Plane Height [m]` was stated as 1.6 m above mean sea level. 

## Plot the shear

### Plot the average shear profile

In [ ]:
average_power_law = bw.Shear.Average(data[wspd_avg_col_names], wspd_avg_col_names_heights)

The above plot shows the average shear profile for this station. The average shear value, alpha, can be extracted from the resulting object as show below.

In [ ]:
print('The average shear alpha value for this station is: ', round(average_power_law.alpha, 2))

### Plot shear by direction sector

For this plot a single wind direction measurement is required. The below function selects the first, which is the top most, wind speed and wind direction measurements. You can of course overwrite this.

In [ ]:
# wspd_main = meas_sta.measurements.wspd_names[0]
# wdir_main = meas_sta.measurements.wdir_names[0]
wspd_main = wspd_avg_col_names[5]
wdir_main = 'lidar_lidar18m_WD_alg_03'
print('The main wind speed measurement for plotting is set to:\t\t{}'.format(wspd_main))
print('The main wind direction measurement for plotting is set to:\t{}'.format(wdir_main))

In [ ]:
shear_by_sec_obj_pw_law = bw.Shear.BySector(data[wspd_avg_col_names], 
                                            wspd_avg_col_names_heights, 
                                            data[wdir_main], calc_method='power_law')

The above plot shows how the shear alpha value can change depending on the direction of the wind flow. Also show is how frequent the wind blows from each direction sector so you can assess if a high shear value from a particular direction has a big impact.

### Shear plot by time of day

In [ ]:
shear_by_time_of_day_obj_pw_law = bw.Shear.TimeOfDay(data[wspd_avg_col_names], wspd_avg_col_names_heights, 
                                                     calc_method='power_law')

The above plot shows the daily profile of the shear alpha value for each calendar month. This shows a more diurnal effect during the summer months thant the winter ones.

An alternative view of this plot is shown below as a heatmap.

In [ ]:
shear_by_time_of_day_obj_pw_law = bw.Shear.TimeOfDay(data[wspd_avg_col_names], wspd_avg_col_names_heights, 
                                                     calc_method='power_law', plot_type='12x24')

## Conclusions

The above work shows how useful the WRA Data Model can be in developing automated workflows along with the use of the brightwind library. In this particular case there were some modifications needed to be performed on the data which was outside of our control. Nevertheless, this demonstrates the possibilities when the format of the timeseries data received can be controlled.

Obviously, what is missing in this workflow are the quality control checks and cleaning of the timeseries data. Therefore, treat any results from this sample workflow with caution